# Autocorrect model

* Identify the misspelled words
* Find strings n edit distance away 
* Filter Candidate - take only words in the vocab
* Word Probability 


This method is first created by "Peter Norvig", 2007

In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd


In [2]:
def process_data(file_name):
    words = []

    with open(file=file_name) as f:
        text = f.read()
    
    text = text.lower()
    words = re.findall(r'\w+', text)
    
    return words


In [3]:
words_l = process_data('shakespeare.txt')
vocab = set(words_l)

len(words_l), len(vocab)


(53614, 6116)

In [4]:
def get_count(words_l):
    word_count_dict = {}
    word_count_dict = Counter(words_l) # dict with unique words and counts 
    return word_count_dict

word_count_dict = get_count(words_l)


In [5]:
def get_probs(word_count_dict):
    m = sum(word_count_dict.values()) # total number of unique words
    probs = {}

    for key in word_count_dict.keys():
        probs[key] = word_count_dict[key] / m

    return probs

In [6]:
probs = get_probs(word_count_dict)

for i in probs: # iterates through keys in the dict
    print(probs[i])
    break

0.0029283396127877045


### Edit Distance
    - Using Dynamic Programming 

We will correct words that are 1 and 2 edit distance away

* Delete (1)
* Switch 
* Replace (2)
* Insert (1)

- *To Compute prob that a certain word is correct given an input*


$$ P(A | B) = \frac{P(B|A)\times P(A)}{P(B)} \tag{Bayes Rule}$$

$P(A|B)$ - Conditional Probability --- posterior probability of A given B

$P(A), P(B)$ --- $prior\space probability$ and $marginal\space probability$ 

In [7]:
def delete_letter(word, verbose=False): # delete one character at a time from the given word 
    delete_l = []
    split_l = []

    for c in range(len(word)):
        split_l.append((word[:c], word[c:]))
    
    for a, b in split_l: # del each letter and store
        delete_l.append(a + b[1:])

    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return delete_l

In [8]:
'''a list of all possible strings with one adjacent charater switched'''

def switch_letter(word, verbose=False):

    switch_l = []
    split_l = []
    
    len_word=len(word)
    for c in range(len_word):
        split_l.append((word[:c],word[c:]))
        
    switch_l = [a + b[1] + b[0] + b[2:] for a,b in split_l if len(b) >= 2]
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 

    return switch_l

In [9]:
switch_word_l = switch_letter(word="ta",
                         verbose=True)

print('\n')

switch_word_l = switch_letter(word="eta",
                         verbose=True)



Input word = ta 
split_l = [('', 'ta'), ('t', 'a')] 
switch_l = ['at']


Input word = eta 
split_l = [('', 'eta'), ('e', 'ta'), ('et', 'a')] 
switch_l = ['tea', 'eat']


In [10]:
'''a list of all possible strings where we replace one letter from the original word'''
def replace_letter(word, verbose=False):
     
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_l = []
    split_l = []

    for c in range(len(word)):
        split_l.append((word[:c], word[c:]))
    
    replace_l = [a + l + (b[1:] if len(b) > 1 else '') for a, b in split_l if b for l in letters]

    # for a, b in split_l:
    #    if b:
    #        for l in letters:
    #            if len(b) > 1:
    #                replace_l.append(a + l + b[1:])
    #           else:
    #               replace_l.append(a+ l)
                    
    replace_set = set(replace_l)
    replace_set.remove(word)

    replace_l = sorted(list(replace_set))

    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   

    return replace_l

In [11]:
replace_l = replace_letter(word='can', verbose=True)

Input word = can 
split_l = [('', 'can'), ('c', 'an'), ('ca', 'n')] 
replace_l ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


In [12]:
len(replace_l), len('can') * 26 

(75, 78)

In [13]:
'''a set of all possible strings with one new letter inserted at every offset'''
def insert_letter(word, verbose=False):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []

    split_l = [(word[:c], word[c:]) for c in range(len(word)+1)]

    insert_l = [a + l + b for a, b in split_l for l in letters]

    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return list(set(insert_l))

In [14]:
insert_l = insert_letter('at', True)

len(insert_l)

Input word at 
split_l = [('', 'at'), ('a', 't'), ('at', '')] 
insert_l = ['aat', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat', 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'att', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz']


76

### Edits

In [15]:
''' A set of words with one possible edit'''
def edit_one_letter(word, allow_switches=True):
    edit_one_set = set()

    edit_one_set.update(delete_letter(word))

    if allow_switches:
        edit_one_set.update(switch_letter(word))
    
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))

    return set(edit_one_set)

In [16]:
tmp_word = "at"
tmp_edit_one_set = edit_one_letter(tmp_word)

tmp_edit_one_l = sorted(list(tmp_edit_one_set))
for i in tmp_edit_one_l:
    print(i, end=', ')

a, aa, aat, ab, abt, ac, act, ad, adt, ae, aet, af, aft, ag, agt, ah, aht, ai, ait, aj, ajt, ak, akt, al, alt, am, amt, an, ant, ao, aot, ap, apt, aq, aqt, ar, art, as, ast, ata, atb, atc, atd, ate, atf, atg, ath, ati, atj, atk, atl, atm, atn, ato, atp, atq, atr, ats, att, atu, atv, atw, atx, aty, atz, au, aut, av, avt, aw, awt, ax, axt, ay, ayt, az, azt, bat, bt, cat, ct, dat, dt, eat, et, fat, ft, gat, gt, hat, ht, iat, it, jat, jt, kat, kt, lat, lt, mat, mt, nat, nt, oat, ot, pat, pt, qat, qt, rat, rt, sat, st, t, ta, tat, tt, uat, ut, vat, vt, wat, wt, xat, xt, yat, yt, zat, zt, 

In [17]:
len(tmp_edit_one_l)

129

In [18]:

def edit_two_letters(word, allow_switches = True):
    
    edit_two_set = set()
    
    edit_one = edit_one_letter(word, allow_switches=allow_switches)
    for w in edit_one: # for all words in the edit one done 
        if w: # if word exists
            edit_two = edit_one_letter(w,allow_switches=allow_switches)
            edit_two_set.update(edit_two) # update with edit 2
    
    return edit_two_set

In [19]:
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))

In [20]:
len(tmp_edit_two_l)

2654

### Suggest Spelling Suggestion

In [21]:

def get_corrections(word, probs, vocab, n=2, verbose = False):

    suggestions = [] # list to store the suggested corrections
    n_best = [] # only give the n best words 

    if word in vocab: # if word in vocab, then it is correct 
        suggestions.append(word)
    else:
        suggestions.extend(edit_one_letter(word).intersection(vocab)) # if only the word in vocab after editing
    
    n_best = sorted(suggestions, key=lambda x: -probs[x])[:n]
    
    n_best = [(s, probs[s]) for s in n_best]

    
    # if verbose: print("entered word = ", word, "\nsuggestions = ", suggestions)

    if verbose:
        print("Entered word = ", word, "\t [ N = ", n, ' ]', "\nSuggestions = ", suggestions)
    
    return n_best


def get_corrections_2edit(word, probs, vocab, n=2, verbose = False):

    suggestions = [] 
    n_best = []

    if word in vocab: # 
        suggestions.append(word)
    else:
        suggestions.extend(edit_two_letters(word).intersection(vocab)) 
    
    n_best = sorted(suggestions, key=lambda x: -probs[x])[:n]
    
    n_best = [(s, probs[s]) for s in n_best]

    
    if verbose:
        print("Entered word = ", word, "\t [ N = ", n, ' ]', "\nSuggestions = ", suggestions)
    
    return n_best

In [22]:
print('\n')
print('===== ONE EDIT DISTANCE =====')

my_word = 'lea' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True) 
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}")

print('\n')
my_word = 'lear' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True)
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}")

print('\n', '===== TWO EDIT DISTANCE =====')
my_word = 'lear' 
tmp_corrections = get_corrections_2edit(my_word, probs, vocab, 4, verbose=True) 
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}")




===== ONE EDIT DISTANCE =====
Entered word =  lea 	 [ N =  2  ] 
Suggestions =  ['led', 'lean', 'leap', 'lead', 'yea', 'let', 'sea', 'leg', 'le', 'plea']
word 0: let
word 1: sea


Entered word =  lear 	 [ N =  2  ] 
Suggestions =  ['sear', 'year', 'clear', 'near', 'lean', 'pear', 'wear', 'hear', 'bear', 'leap', 'lead', 'rear', 'liar', 'ear', 'learn', 'dear', 'tear', 'fear']
word 0: hear
word 1: dear

 ===== TWO EDIT DISTANCE =====
Entered word =  lear 	 [ N =  4  ] 
Suggestions =  ['pears', 'lead', 'ear', 'fear', 'heir', 'ears', 'seat', 'meat', 'fears', 'wear', 'lay', 'beard', 'learn', 'dar', 'tear', 'heat', 'clears', 'star', 'earl', 'err', 'years', 'let', 'real', 'plead', 'leaner', 'heart', 'lent', 'her', 'year', 'clear', 'pear', 'left', 'dear', 'leg', 'dead', 'deal', 'near', 'war', 'lest', 'leads', 'mean', 'scar', 'ceas', 'leave', 'leash', 'law', 'lean', 'beam', 'led', 'deer', 'lease', 'altar', 'sea', 'lap', 'head', 'bar', 'seal', 'hears', 'par', 'roar', 'read', 'rear', 'le', 'swea

In [25]:
from ipywidgets import widgets

def get_corrections_interactive(btn):
    suggestions = []
    if edit_dropdown.value == "1 edit":
        suggestions = get_corrections(word_input.value, probs, vocab, n=4)
    elif edit_dropdown.value == "2 edit":
        suggestions = get_corrections_2edit(word_input.value, probs, vocab, n=4)
    else:
        print("Invalid choice of edit distance.")

    print(f"Top 4 suggestions based on {edit_dropdown.value}: {[word_prob[0] for word_prob in suggestions[:4]]}")

# Dropdown widget for edit distance
edit_dropdown = widgets.Dropdown(
    options=["1 edit", "2 edit"],
    description='Edit Distance:',
    disabled=False
)

# Text input widget for word
word_input = widgets.Text(
    placeholder='Enter a word',
    description='Word:',
    disabled=False
)

# Button widget for autocorrect
autocorrect_button = widgets.Button(description="Autocorrect")
autocorrect_button.on_click(get_corrections_interactive)

display(word_input)
display(edit_dropdown)
display(autocorrect_button)



Text(value='', description='Word:', placeholder='Enter a word')

Dropdown(description='Edit Distance:', options=('1 edit', '2 edit'), value='1 edit')

Button(description='Autocorrect', style=ButtonStyle())

## Minimum Edit Distance

To evaluate the similarity between two strings. Shortest path to go from a word to another word.

In [59]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    '''
    Input: 
        source: a string corresponding to the string you are starting with
        target: a string corresponding to the string you want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    '''
    m = len(source) 
    n = len(target) 
    #initialize cost matrix with zeros and dimensions (m+1,n+1) 
    D = np.zeros((m+1, n+1), dtype=int) 
    
    # D[0, 0] is 0
    # Fill in column 0, from row 1 to row m
    for row in range(1,m+1): 
        D[row,0] = D[row-1,0] + del_cost
        
    # Fill in row 0, for all columns from 1 to n
    for col in range(1,n+1): 
        D[0,col] = D[0,col-1] + ins_cost
        
    for row in range(1,m+1): 
        for col in range(1,n+1):
            
            r_cost = rep_cost
            
            # Check to see if source character at the previous row matches the target character at the previous column, 
            if source[row-1] == target[col-1]:
                # Update the replacement cost to 0 if source and target are the same
                r_cost = 0
                
            # Update the cost at row, col based on previous entries in the cost matrix
            # Refer to the equation calculate for D[i,j] (the minimum of three calculated costs)
            D[row,col] = min([D[row-1,col]+del_cost, D[row,col-1]+ins_cost, D[row-1,col-1]+r_cost])
          
    # Set the minimum edit distance with the cost found at row m, column n
    med = D[m,n]

    return D, med

In [60]:

source =  'play'
target = 'stay'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  4 

   #  s  t  a  y
#  0  1  2  3  4
p  1  2  3  4  5
l  2  3  4  5  6
a  3  4  5  4  5
y  4  5  6  5  4


In [61]:
source = "eer"
targets = edit_one_letter(source,allow_switches = False)  #disable switches since min_edit_distance does not include them
for t in targets:
    _, min_edits = min_edit_distance(source, t,1,1,1)  # set ins, del, sub costs all to one
    if min_edits != 1: print(source, t, min_edits)

In [64]:
source = "eer"
targets = edit_two_letters(source,allow_switches = False) #disable switches since min_edit_distance does not include them
for t in targets:
    _, min_edits = min_edit_distance(source, t,1,1,1)  # set ins, del, sub costs all to one
    if min_edits != 2 and min_edits != 1: print(source, t, min_edits)

eer eer 0
